**Dimensionality Reductiont**

In [1]:
from IPython.display import Image
Image("./Projection.png")

In [6]:
from sklearn.datasets import fetch_mldata
mnist = fetch_mldata('MNIST original')

In [7]:
from mnist import MNIST

ModuleNotFoundError: ignored

Split Data into Training and Test Sets

In [8]:
from sklearn.model_selection import train_test_split
# test_size: what proportion of original data is used for test set
train_img, test_img, train_lbl, test_lbl = train_test_split( mnist.data, mnist.target, test_size=1/7.0, random_state=0)

NameError: ignored

Standardize the Data

In [0]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
# Fit on training set only.
scaler.fit(train_img)
# Apply transform to both the training set and the test set.
train_img = scaler.transform(train_img)
test_img = scaler.transform(test_img)

Import and Apply PCA

In [0]:
from sklearn.decomposition import PCA
# Make an instance of the Model
pca = PCA(.95)

In [0]:
pca.fit(train_img)

Apply the mapping (transform) to both the training set and the test set.

In [0]:
train_img = pca.transform(train_img)
test_img = pca.transform(test_img)

Apply Logistic Regression to the Transformed Data

In [0]:
from sklearn.linear_model import LogisticRegression

# all parameters not specified are set to their defaults
# default solver is incredibly slow which is why it was changed to 'lbfgs'
logisticRegr = LogisticRegression(solver = 'lbfgs')

logisticRegr.fit(train_img, train_lbl)

# Predict for One Observation (image)
logisticRegr.predict(test_img[0].reshape(1,-1))

In [0]:
# Predict for One Observation (image)
logisticRegr.predict(test_img[0:10])

Measuring Model Performance

In [0]:
logisticRegr.score(test_img, test_lbl)

Manifold Learning

In [0]:
from IPython.display import Image
Image("./mainfoldlearning.png")

PCA

The following Python code uses NumPy’s svd() function to obtain all the principal components of the
training set, then extracts the first two PCs:

In [2]:
X_centered = X - X.mean(axis=0)
U, s, V = np.linalg.svd(X_centered)
c1 = V.T[:, 0]
c2 = V.T[:, 1]

NameError: ignored

The following Python code projects the training set onto the plane defined by the first two principal
components:

In [0]:
W2 = V.T[:, :2]
X2D = X_centered.dot(W2)

In [0]:
from sklearn.decomposition import PCA
pca = PCA(n_components = 2)
X2D = pca.fit_transform(X)

print(pca.explained_variance_ratio_)


Choosing the Right Number of Dimensions

In [0]:
pca = PCA()
pca.fit(X)
cumsum = np.cumsum(pca.explained_variance_ratio_)
d = np.argmax(cumsum >= 0.95) + 1

In [0]:
pca = PCA(n_components=0.95)
X_reduced = pca.fit_transform(X)

PCA for Compression

In [0]:
pca = PCA(n_components = 154)
X_mnist_reduced = pca.fit_transform(X_mnist)
X_mnist_recovered = pca.inverse_transform(X_mnist_reduced)


Incremental PCA

In [0]:
from sklearn.decomposition import IncrementalPCA
n_batches = 100
inc_pca = IncrementalPCA(n_components=154)
for X_batch in np.array_split(X_mnist, n_batches):
inc_pca.partial_fit(X_batch)
X_mnist_reduced = inc_pca.transform(X_mnist)

Alternatively, you can use NumPy’s memmap class

In [0]:
X_mm = np.memmap(filename, dtype="float32", mode="readonly", shape=(m, n))
batch_size = m // n_batches
inc_pca = IncrementalPCA(n_components=154, batch_size=batch_size)
inc_pca.fit(X_mm)

Randomized PCA

In [0]:
rnd_pca = PCA(n_components=154, svd_solver="randomized")
X_reduced = rnd_pca.fit_transform(X_mnist)

Kernel PCA

In [0]:
from sklearn.decomposition import KernelPCA
rbf_pca = KernelPCA(n_components = 2, kernel="rbf", gamma=0.04)
X_reduced = rbf_pca.fit_transform(X)

Selecting a Kernel and Tuning Hyperparameters

In [0]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
clf = Pipeline([
("kpca", KernelPCA(n_components=2)),
("log_reg", LogisticRegression())
])
param_grid = [{
"kpca__gamma": np.linspace(0.03, 0.05, 10),
"kpca__kernel": ["rbf", "sigmoid"]
}]
grid_search = GridSearchCV(clf, param_grid, cv=3)
grid_search.fit(X, y)

The best kernel and hyperparameters are then available through the best_params_ variable:


In [0]:
print(grid_search.best_params_)

In [0]:
rbf_pca = KernelPCA(n_components = 2, kernel="rbf", gamma=0.0433,
fit_inverse_transform=True)
X_reduced = rbf_pca.fit_transform(X)
X_preimage = rbf_pca.inverse_transform(X_reduced)

In [0]:
from sklearn.metrics import mean_squared_error
mean_squared_error(X, X_preimage)

In [0]:
LLE

In [0]:
from sklearn.manifold import LocallyLinearEmbedding
lle = LocallyLinearEmbedding(n_components=2, n_neighbors=10)
X_reduced = lle.fit_transform(X)